In [79]:
# Import libraries
import pandas as pd
import google.auth
from googleapiclient.discovery import build
from dotenv import load_dotenv
import os

# Import api key and video id
load_dotenv()
api_key = os.getenv('API_KEY')
video_id = os.getenv('VIDEO_ID')

In [81]:
def video_comments(video_id):
	# Create a list to store comments
	comments_data = []

	# Create YouTube resource object
	youtube = build('youtube', 'v3', developerKey=api_key)

	# Retrieve YouTube video results
	video_response=youtube.commentThreads().list(
	part='snippet,replies',
	videoId=video_id
	).execute()

	# Iterate through video results
	while video_response:
	
		# Extract information from each object
		for item in video_response['items']:
		
			# Extract comments
			comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
			
			# Count number of replies on each comment
			replycount = item['snippet']['totalReplyCount']

			# Save replies if there is one
			if replycount>0:
				replies = [reply['snippet']['textDisplay'] for reply in item['replies']['comments']]
			else:
				replies = []

			comments_data.append({'Comment': comment, 'Replies': replies})

		# Repeat for next page
		if 'nextPageToken' in video_response:
			video_response = youtube.commentThreads().list(
					part = 'snippet,replies',
					videoId = video_id,
					pageToken = video_response['nextPageToken']
				).execute()
		else:
			break

	# Create DataFrame containing comments and replies
	df = pd.DataFrame(comments_data)

	# Create a column counting number of replies for that comment
	df['Reply_Count'] = df['Replies'].apply(len)

	return df

# Create DataFrame containing comments and replies from video
comments_df = video_comments(video_id)

In [82]:
print('Comments: {}, Replies: {}, Total: {}'.format(len(comments_df), comments_df['Reply_Count'].sum(), len(comments_df)+comments_df['Reply_Count'].sum()))
comments_df.head()


Comments: 216, Replies: 112, Total: 328


,Comment,Replies,Reply_Count
0,Another great 6 nations contest. Hard fought a...,[],0
1,Scotland deserved a win! These TMO guys are us...,[],0
2,No try. End of story. Don&#39;t blame the ref....,[],0
3,Scotland robbed,[],0
4,No the Scots understands what South Africa has...,[],0
